In [1]:
import time

import datetime
import pytz
import calendar
import itertools
import pandas as pd

In [2]:
import datetime
import sys

import networkx as nx
import numpy as np
import os
import pandas as pd
import plotly.offline as py
import pytz

sys.path.insert(0, '../')
# import utils
# from utils import plotly_figure

import itertools
import xbos_services_getter as xsg

# Check that correct data is given

In [66]:
def check_data(data, start, end, window):
    """Checks if data has right times and does not contain any Nan values. 
    
    :param data: pd.df or pd.series
    :param start: datetime (timezone aware)
    :param end: datetime (timezone aware)
    :param window: (string)
    :return: str"""
    window = xsg.get_window_in_sec(window)
    if not isinstance(data, pd.DataFrame) and not isinstance(data, pd.Series):
        return "Is not a pd.DataFrame/pd.Series"
    if (start not in data.index) or ((end - datetime.timedelta(seconds=window)) not in data.index):
        return "Does not have valid start or/and end time."
    if data.isna().values.any():
        return "Nan values in data."
    time_diffs = data.index.to_series(keep_tz=True).diff()
    if (time_diffs.shape[0] > 1) and ((time_diffs.min() != time_diffs.max()) or (time_diffs.min().seconds != window)):
        return "Missing rows or/and bad time frequency."
    return None

def check_data_zones(zones, data_dict, start, end, window):
    for zone in zones:
        if zone not in data_dict:
            return "Is missing zone " + zone
        err = check_data(data_dict[zone], start, end, window)
        if err is not None:
            return err
    return None


# Define OptimizerParent
The optimizer parent can be used accross optimizers. It makes it easy to get the relevant data for training and testing purposes. 

Assumes that an Optimizer instance is created for every time the optimization is called.

In [67]:
class OptimizerParent:
    def __init__(self, building, zones, start, end, window, non_controllable_data={}):
        
        self.start = start
        self.unix_start = start.timestamp() * 1e9
        self.end = end
        self.unix_end = end.timestamp() * 1e9
        self.window = window  # timedelta
        
        self.building = building
        self.zones = zones 
        
        if non_controllable_data is None:
            non_controllable_data = {}
        # TODO add error checking. check that the right zones are given in non_controllable_data and that the start/end/window are right. 
        
        # Documentation: All data here is in timeseries starting exactly at start and every step corresponds to one 
        # interval. The end is not inclusive.

        # temperature band
        temperature_band_stub = xsg.get_temperature_band_stub()
        
        if "comfortband" not in non_controllable_data:
            self.comfortband = {iter_zone: xsg.get_comfortband(temperature_band_stub, self.building, iter_zone, self.start, self.end, self.window)
                                  for iter_zone in self.zones}
        else:
            self.comfortband = non_controllable_data["comfortband"]
        err = check_data_zones(self.zones, self.comfortband, start, end, window)
        if err is not None:
            raise Exception("Bad comfortband given. " + err)
            
        if "do_not_exceed" not in non_controllable_data:
            self.do_not_exceed = {iter_zone: xsg.get_do_not_exceed(temperature_band_stub, self.building, iter_zone, self.start, self.end, self.window)
                                  for iter_zone in self.zones}
        else:
            self.do_not_exceed = non_controllable_data["do_not_exceed"]
        err = check_data_zones(self.zones, self.do_not_exceed, start, end, window)
        if err is not None:
            raise Exception("Bad DoNotExceed given. " + err)
        
        # occupancy
        if non_controllable_data is None or "occupancy" not in non_controllable_data:
            occupancy_stub = xsg.get_occupancy_stub()
            self.occupancy = {iter_zone: xsg.get_occupancy(occupancy_stub, self.building, iter_zone, self.start, self.end, self.window)
                             for iter_zone in self.zones}
        else:
            self.occupancy = non_controllable_data["occupancy"]
        err = check_data_zones(self.zones, self.occupancy, start, end, window)
        if err is not None:
            raise Exception("Bad occupancy given. " + err)
        
        # outdoor temperatures
        if "outdoor_temperature" not in non_controllable_data:
            outdoor_historic_stub = xsg.get_outdoor_historic_stub()
            self.outdoor_temperature = xsg.get_outdoor_temperature_historic(outdoor_historic_stub, self.building,
                                                                           self.start, self.end, self.window)
        err = check_data(self.outdoor_temperature, start, end, window)
        if err is not None:
            raise Exception("Bad outdoor temperature given. " + err)
#         outdoor_prediction_channel = grpc.insecure_channel(OUTSIDE_PREDICTION)
#         outdoor_prediction_stub = outdoor_temperature_prediction_pb2_grpc.OutdoorTemperatureStub(outdoor_prediction_channel)

#         self.outdoor_temperatures = get_outside_temperature(
#             outdoor_historic_stub, outdoor_prediction_stub, self.building, self.start, self.end, self.window)

        # discomfort channel 
        self.discomfort_stub = xsg.get_discomfort_stub()
        
        # HVAC Consumption TODO ERROR CHECK?
        hvac_consumption_stub = xsg.get_hvac_consumption_stub()
        self.hvac_consumption = {iter_zone: xsg.get_hvac_consumption(hvac_consumption_stub, building, iter_zone) 
                                 for iter_zone in self.zones}
        
        # TODO Prices
        


# Shortest Path (MPC) Implementation

Here the MPC is implemented. It inherits the data methods from OptimizerParent.

In [114]:

# TODO Demand charges right now assume constant demand charge throughout interval. should be easy to extend
# TODO but need to keep in mind that we need to then store the cost of demand charge and not the consumption
# TODO in the graph, since we actually only want to minimize cost and not consumption.

# TODO if there is no feasible solution in which we can stay within a safety region –– revert to standard control and force it to go within saftey region. 

class Node:
    """
    # this is a Node of the graph for the shortest path
    """

    def __init__(self, temperatures, timestep):
        self.temperatures = temperatures
        self.timestep = timestep

    def __hash__(self):
        return hash((' '.join(str(e) for e in self.temperatures), self.timestep))

    def __eq__(self, other):
        return isinstance(other, self.__class__) \
               and self.temperatures == other.temperatures \
               and self.timestep == other.timestep

    def __repr__(self):
        return "{0}-{1}".format(self.timestep, self.temperatures)


class MPC(OptimizerParent):
    """MPC Optimizer. 
    No Demand Charges and Two Stage actions implemented."""
    
    def __init__(self, building, zones, start, end, window, lambda_val,
                 root=Node([75], 0), non_controllable_data=None, debug=False):
        """
        initialize instance variables
        
        :param building: (str) building name
        :param zones: [str] zone names
        :param start: (datetime timezone aware) 
        :param end: (datetime timezone aware) 
        :param window: (str) the interval in which to split the data.
        :param lambda_val: (float) lambda value for opjective function
        :param root: (Node) the node at which to start

        """
        super().__init__(building, zones, start, end, window, non_controllable_data)

        self.root = root
        self.lambda_val = lambda_val
        self.debug = debug

        self.g = nx.DiGraph()  # [TODO:Changed to MultiDiGraph... FIX print]
        
    def safety_check(self, node):
        for iter_zone in self.zones:
            curr_temperature = node.temperatures[iter_zone]
            curr_safety = self.do_not_exceed[iter_zone].iloc[node.timestep]
            if not (curr_safety["t_low"] <= curr_temperature <= curr_safety["t_high"]):
                return False
        return True
    
    def timestep_to_datetime(self, timestep):
        return start + timestep*datetime.timedelta(seconds=xsg.get_window_in_sec(self.window))

    # the shortest path algorithm
    def shortest_path(self, root):
        """
        Creates the graph using DFS and calculates the shortest path
    
        :param root: node being examined right now and needs to be added to graph. 
        
        :return: root Node if root added else return None. 
        
        """
        
        if root is None:
            return None
        
        if root in self.g:
            return root
                
        # stop if node is past predictive horizon
        if self.timestep_to_datetime(root.timestep) >= self.end:
            self.g.add_node(root, objective_cost=0, best_action=None, best_successor=None) # no cost as leaf node
            return root
        
        # check if valid node
        if not self.safety_check(root):
            return None
        
        self.g.add_node(root, objective_cost=np.inf, best_action=None, best_successor=None)

        # creating children, adding corresponding edge and updating root's objective cost
        for action in itertools.product([xsg.NO_ACTION, xsg.HEATING_ACTION, xsg.COOLING_ACTION], 
                                       repeat=len(self.zones)):
                
            # TODO Compute temperatures properly
            temperatures = {}
            zone_action = {}
            for i in range(len(self.zones)):
                zone_action[self.zones[i]] = action[i]
                temperatures[self.zones[i]] = root.temperatures[self.zones[i]] + \
                                                    1 * (action[i]==1) - 1*(action[i]==2)
                    
            # Create child node and call the shortest_path recursively on it
            child_node = Node(
                temperatures=temperatures,
                timestep=root.timestep + 1
                )

            child_node = self.shortest_path(child_node)
            if child_node is None:
                continue

            # get discomfort across edge
            discomfort = {}
            for iter_zone in self.zones:
                curr_comfortband = self.comfortband[iter_zone].iloc[root.timestep]
                curr_occupancy = self.occupancy[iter_zone].iloc[root.timestep]
                average_edge_temperature = (root.temperatures[iter_zone] + child_node.temperatures[iter_zone])/2.

                discomfort[iter_zone] = xsg.get_discomfort(
                    self.discomfort_stub, self.building, average_edge_temperature,
                    curr_comfortband["t_low"], curr_comfortband["t_high"], 
                    curr_occupancy)

            # Get consumption across edge
            price = 1  # self.prices.iloc[root.timestep] TODO also add right unit conversion, and duration
            consumption_cost = {self.zones[i]: price * self.hvac_consumption[self.zones[i]][action[i]] 
                               for i in range(len(self.zones))}

            # add edge 
            self.g.add_edge(root, child_node, action=zone_action, discomfort=discomfort, consumption_cost=consumption_cost)

            # update root node to contain the best child.
            total_edge_cost = ((1 - self.lambda_val) * (sum(consumption_cost.values()))) + (
                self.lambda_val * (sum(discomfort.values())))
            
            objective_cost = self.g.node[child_node]["objective_cost"] + total_edge_cost

            if objective_cost < self.g.node[root]["objective_cost"]:
                self.g.node[root]["objective_cost"] = objective_cost
                self.g.node[root]["best_action"] = zone_action
                self.g.node[root]["best_successor"] = child_node
                
        return root
 
    def reconstruct_path(self, root):
        """
        Util function that reconstructs the best action path
        Parameters
        ----------
        graph : networkx graph

        Returns
        -------
        List
        """
        graph = self.g 
        
        if root not in self.g:
            raise Exception("Root does not exist in MPC graph.")

        
        path = [root]

        while graph.node[root]['best_successor'] is not None:
            root = graph.node[root]['best_successor']
            path.append(root)

        return path
    
#     def g_plot(self, zone):
#         try:
#             os.remove('mpc_graph_' + zone + '.html')
#         except OSError:
#             pass

#         fig = plotly_figure(self.advise_unit.g, path=self.path)
#         py.plot(fig, filename='mpc_graph_' + zone + '.html', auto_open=False)


# Preliminary Testing of MPC

In [115]:
names_stub = xsg.get_building_zone_names_stub()
all_names = xsg.get_all_buildings_zones(names_stub)

In [116]:
all_names["avenal-animal-shelter"]

['hvac_zone_shelter_corridor']

In [117]:
end = datetime.datetime.utcnow().replace(tzinfo=pytz.utc)
start =  end - datetime.timedelta(hours=4)
print(start)
start = start.replace(microsecond=0)
print(start)

2019-04-11 23:45:57.707927+00:00
2019-04-11 23:45:57+00:00


In [118]:

end = datetime.datetime.utcnow().replace(tzinfo=pytz.utc)
end = end.replace(microsecond=0)
start =  end - datetime.timedelta(hours=4)

print(start)
print(start.timestamp())
bldg = "avenal-animal-shelter"
zone = "hvac_zone_shelter_corridor"
window = "15m"

op = MPC(bldg, [zone], start, end, window, 0.995)

2019-04-11 23:45:58+00:00
1555026358.0


In [119]:
# Run shortest path
root = Node({zone: 75}, 0)
op.occupancy[zone][:] = 1

In [120]:
print(op.do_not_exceed)

{'hvac_zone_shelter_corridor':                           t_high t_low
2019-04-11 23:45:58+00:00     85    68
2019-04-12 00:00:58+00:00     85    68
2019-04-12 00:15:58+00:00     85    68
2019-04-12 00:30:58+00:00     85    68
2019-04-12 00:45:58+00:00     85    68
2019-04-12 01:00:58+00:00     85    68
2019-04-12 01:15:58+00:00     85    68
2019-04-12 01:30:58+00:00     85    68
2019-04-12 01:45:58+00:00     85    68
2019-04-12 02:00:58+00:00     85    68
2019-04-12 02:15:58+00:00     85    68
2019-04-12 02:30:58+00:00     85    68
2019-04-12 02:45:58+00:00     85    68
2019-04-12 03:00:58+00:00     85    68
2019-04-12 03:15:58+00:00     85    68
2019-04-12 03:30:58+00:00     85    68}


In [121]:
root = op.shortest_path(root)
print(root)

0-{'hvac_zone_shelter_corridor': 75}


In [122]:
print(op.g.node[root]["best_action"])

{'hvac_zone_shelter_corridor': 0}


In [96]:
op.safety_check(root)

True

In [97]:
op.reconstruct_path(root)

[0-{'hvac_zone_shelter_corridor': 75},
 1-{'hvac_zone_shelter_corridor': 75},
 2-{'hvac_zone_shelter_corridor': 75},
 3-{'hvac_zone_shelter_corridor': 75},
 4-{'hvac_zone_shelter_corridor': 75},
 5-{'hvac_zone_shelter_corridor': 75},
 6-{'hvac_zone_shelter_corridor': 75},
 7-{'hvac_zone_shelter_corridor': 75},
 8-{'hvac_zone_shelter_corridor': 75},
 9-{'hvac_zone_shelter_corridor': 75},
 10-{'hvac_zone_shelter_corridor': 75},
 11-{'hvac_zone_shelter_corridor': 75},
 12-{'hvac_zone_shelter_corridor': 75},
 13-{'hvac_zone_shelter_corridor': 75},
 14-{'hvac_zone_shelter_corridor': 75},
 15-{'hvac_zone_shelter_corridor': 75},
 16-{'hvac_zone_shelter_corridor': 75}]

In [126]:
class SimulationMPC():
    
    def __init__(self, building, zones, lambda_val, start, end, forecasting_horizon, window, tstats, non_contrallable_data=None):
        self.building = building
        self.zones = zones
        self.lambda_val = lambda_val
        
        self.start = start # datetime timezone aware
        self.end = end # datetime timezone aware
        self.forecasting_horizon = forecasting_horizon# (str)
        self.window = window# (str, between optimizations)
        self.tstats = tstats# dictionary of simulator object with key zone. has functions: current_temperature, next_temperature(action)
        
        self.non_controllable_data = non_contrallable_data # dictionary should have keys [outdoor_temperature, occupancy, prices, comfortband, do_not_exceed] and be from start to end + forecasting horizon.
        
        self.current_time = start 
        
        self.actions = []
        
        
    def get_time_step(curr_time):
        pass
        
    def step(self):
        
        # call
        start_mpc = self.current_time
        end_mpc = self.current_time + datetime.timedelta(seconds=xsg.get_window_in_sec(self.forecasting_horizon))
        op = MPC(self.building, self.zones, start_mpc, end_mpc, window, self.lambda_val, debug=False)
        
        root = Node({iter_zone: tstats[iter_zone].temperature for iter_zone in self.zones}, 0)
        
        root = op.shortest_path(root)
        best_action = op.g.node[root]["best_action"]
        print(best_action)
        
        
        # given the action, update simulation of temperature. 
        # increment time
        
        
        return root

    def run(self):
        while self.current_time < self.end: 
            self.step()
        


In [127]:

end = datetime.datetime.utcnow().replace(tzinfo=pytz.utc)
end = end.replace(microsecond=0)
start =  end - datetime.timedelta(hours=24)

print(start)
print(start.timestamp())
building = "avenal-animal-shelter"
zones = ["hvac_zone_shelter_corridor"]
window = "15m"
forecasting_horizon = "4h"
lambda_val = 0.995
tstats = {iter_zone: Tstat(75) for iter_zone in zones}
simulation = SimulationMPC(building, zones, lambda_val, start, end, forecasting_horizon, window, tstats)
simulation.step()

2019-04-11 03:46:41+00:00
1554954401.0
{'hvac_zone_shelter_corridor': 0}


0-{'hvac_zone_shelter_corridor': 75}

In [85]:
class Tstat:
    def __init__(self, temperature):
        self.temperature = temperature
        
    def next_temperature(self, action):
        self.temperature += 1 + 1 * (action==1) - 1*(action==2) + np.random.normal(0, 0.6)
        return self.temperature
    

In [3]:



    
# Convert index to tz_naive so that it can be joined with meter data and oat data
# data2 = data2.tz_localize(None)

# data2.head()

SyntaxError: invalid character in identifier (<ipython-input-3-10484913f3ed>, line 20)

In [14]:
s = '2018-01-01T00:00:00Z'
e = '2018-01-15T00:00:00Z'
site = ['ciee']
window = '15m'
point_type = 'Building_Electric_Meter'
agg = 'MEAN'
window = '15m'

s1 = datetime.datetime(2018, 1, 1, 0, 0, 0, 0, pytz.UTC)
e1 = datetime.datetime(2018, 1, 1, 0, 0, 0, 0, pytz.UTC)

meter_data_historical_stub = xsg.get_meter_data_historical_stub()


In [11]:
data2 = xsg.get_meter_data_historical(meter_data_stub=meter_data_historical_stub, 
                                                      bldg=site, 
                                                      start=s1, 
                                                      end=e1, 
                                                      point_type=point_type, 
                                                      aggregate=agg, 
                                                      window=window)

SyntaxError: invalid character in identifier (<ipython-input-11-01762378e869>, line 2)

In [15]:
xsg.get_meter_data_historical(meter_data_historical_stub, site, s1, e1, point_type, agg, window)

_Rendezvous: <_Rendezvous of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Exception calling application: 'NoneType' object has no attribute 'columns'"
	debug_error_string = "{"created":"@1555111166.153281000","description":"Error received from peer","file":"src/core/lib/surface/call.cc","file_line":1017,"grpc_message":"Exception calling application: 'NoneType' object has no attribute 'columns'","grpc_status":2}"
>